# Algunos de los topicos a tratar

* Representacion como vector de un sistema de n qubits.
* 1 qubit operations
* 2 qubits operations (en particular, interacción de Ising)

In [4]:
push!(LOAD_PATH, ".");
using quantum
names(quantum)

16-element Array{Symbol,1}:
 :P_Orthogonal  
 :P_Unitary     
 :apply_ising!  
 :apply_kick!   
 :apply_unitary!
 :base_state    
 :projector     
 :quantum       
 :random_state  
 :sigma_x       
 :sigma_y       
 :sigma_z       
 :sigmas        
 :staircase     
 :testbit       
 :unfolding     

In [5]:
Pkg.update();
Pkg.add("LsqFit");

INFO: Updating METADATA...
INFO: Computing changes...
INFO: No packages to install, update or remove


In [6]:
psi=random_state(2);
θ=2*π*rand();

θm=exp(-im*θ);
θp=conj(θm);

Comenzaremos recordando como se ve el operador de evolución correspondiente a un campo magnético, de magnitud $\theta$ y dirección $z$:
\begin{equation}
\text{exp}(-i \theta \sigma_z) =
\begin{pmatrix}
e^{-i\theta} & 0\\
0& e^{i\theta}
\end{pmatrix}
\end{equation}
En la siguiente linea mostramos explícitamente esta relación y de paso mostramos un comando útil: 
~~~
expm
~~~
que calcula la exponencial de una matriz.

In [7]:
@show norm(expm(-im*θ*sigma_z)-[[θm, 0] [0, θp]]);

norm(expm(-im * θ * sigma_z) - [[θm,0] [0,θp]]) = 1.2412670766236366e-16


Ahora mostramos que aplicar la matriz, es lo mismo que operar por vectores componente a componente. Es decir, la ecuacion trivial
\begin{equation}
\text{exp}(-i\theta \sigma_z)\psi=
\begin{pmatrix}
e^{-i\theta} \psi_1 \\
e^{i\theta} \psi_0 \\
\end{pmatrix}
\end{equation}
Podemos ver como afecta componente por componente. 

In [8]:
@show norm(expm(-im*θ*sigma_z)*psi-[psi[1]*θm, psi[2]*θp]);

norm(expm(-im * θ * sigma_z) * psi - [psi[1] * θm,psi[2] * θp]) = 1.6653345369377348e-16


Para definir estados de qubits, que por ende tienen dimensión $2^N$, donde $N$ es el numero de qubits, es útil usar notación binaria:

Consideremos el espacio de Hilbert de $n$ qubits. La base computational está formada por secuencias de 0s y 1s. Es claro entonces que la dimensión del espacio de Hilbert es  $2^N$. Para 3 qubits, la base computacional sería
\begin{align}
&|000\rangle\\
&|001\rangle\\
&|010\rangle\\
&|011\rangle\\
&|100\rangle\\
&|101\rangle\\
&|110\rangle\\
&|111\rangle.
\end{align}
Para codificar esto de manera conveniente, usar notación binaria. Recordemos, por ejemplo que 
\begin{equation}
(5)_{10} = (101)_2
\end{equation}
de tal manera que podemos resumir 
\begin{equation}
|101\rangle \equiv |5\rangle.
\end{equation}
Así, nuestra base computacional será traducida a 
\begin{align}
|000\rangle \to |0\rangle \\
|001\rangle \to |1\rangle \\
|010\rangle \to |2\rangle \\
|011\rangle \to |3\rangle \\
|100\rangle \to |4\rangle \\
|101\rangle \to |5\rangle \\
|110\rangle \to |6\rangle \\
|111\rangle \to |7\rangle .
\end{align}

Luego, por simplicidad, utilizando la base computacional como nuestra base "canonica", la sustitución es simple:
$|000\rangle \to \left( \begin{array}{c} 1 \\ \vdots \\ 0 \end{array} \right)$, $|001\rangle \to \left( \begin{array}{c} 0 \\ 1 \\ \vdots \end{array} \right)$, etc.

Es decir, la componente que lleva el "1" es precisamente la representacion decimal del estado (comenzando a contar desde cero porsupuesto). Sin embargo en julia los vectores se indexan comenzando de 1, por lo que no tenemos la indexacion tan facil. Sumando uno a los indices solucionamos el problema. 


# Operaciones de un qubit

Al realizar operaciones de un qubit, no estamos mezclando elementos de cualquier tipo. Noten que si hacemos una operacion sobre un solo qubit, mezclamos solo algunos elementos entre si. 

Numeremos los qubits de derecha a izquierda, como numeramos los dígitos en notación decimal. Es decir, en 
\begin{equation}
97834
\end{equation}
el digito 0 es el 4, el digito 1 es el 3, el digito 2 es el 8, etc. Esto ademas resulta conveniente pues podemos escribir
\begin{equation}
97834=4 \times 10^0 + 
3\times 10^1 +
8\times 10^2 +
7\times 10^3 +
9\times 10^4.
\end{equation}
De la misma manera, 
\begin{equation}
(5)_{10}=(101)_2 
=  1\times 2^0+ 0\times 2^1+ 1\times 2^2
\end{equation}


Ahora veamos que si hacemos una operación sobre el qubit 1, vamos a mezclar los estados por pares. Por ejemplo, al aplicar la operación de un solo qubit $u$ sobre el qubit 2, tendremos 
\begin{equation}
u_2 |i_3 i_2 i_1 i_0 \rangle =
1_3 \otimes u_2 
\otimes 1_1 
\otimes 1_0
|i_3 i_2 i_1 i_0 \rangle =
|i_3 \rangle \otimes  
(u |i_2 \rangle) \otimes  
|i_1 \rangle \otimes  
|i_0 \rangle =
\alpha |i_3 i_2 i_1 i_0 \rangle +
\beta |i_3 \bar i_2 i_1 i_0 \rangle.
\end{equation}
Es decir estamos mezclando solo dos estados. Por ejemplo, cualquier operación de un solo qubit, sobre el qubit 1 va a mezclar 
\begin{equation}
|101\rangle 
\end{equation}
con
\begin{equation}
|111\rangle .
\end{equation}




Para refrescar la memoria y tener presente (i) la forma en que representamos NOSOTROS y (2) la COMPU los numeros enteros, usamos las funciones 
~~~
bits
~~~
que nos da la representación binaria de los objetos, y 
~~~
bin
~~~
que nos da la forma en que un entero es representado en base 2. 

In [ ]:
n=4;
for i = 0:2^n-1
    @show bits(i),bin(i,n),i
end

Vamos a construir la rutina para aplicar una operación de un qubit a un estado codificado como se explicó arriba. Consideraremos un estado de $n$ qubits. Este estado tendrá una longitud de $2^n$. Para extraer el número de qubits y evitar meterlo explicitamente en la rutina (redundancia en programación es una muy mala práctica) usamos la función 

In [ ]:
?trailing_zeros

Esta función me dá el numero de zeros a la derecha del ultimo 1:

In [ ]:
n=3;
for i = 0:2^n-1
    @show bin(i,n),trailing_zeros(i)
end

Si el número es una potencia de 2, nos da el valor del logaritmo en base 2, de la misma manera que contar el numero de ceros en el numero 1000 nos dice que el logaritmo, en base 10, de 1000 es 3. 

El siguiente paso es ver que tendremos unos qubits a la izquierda del qubit afectado y otros a la derecha. En el caso de 5 qubits, supongamos que vamos a operar sobre el qubit 2 (el de la mitad). 

Mejor les explico en vivo y en directo. 

In [ ]:
"""
    apply_unitary!(psi, u, target_qubit)

This function applies efficiently an arbitrary unitary "u" to the target qubit, and modifies the state psi. 

"""
function apply_unitary!(psi, u, target_qubit)
    number_of_qubits = trailing_zeros(length(psi))
    end_outer_counter = 2^(number_of_qubits-target_qubit-1)-1
    for counter_left_bits = 0:end_outer_counter
        number_left=counter_left_bits<< (target_qubit+1)
        end_right_counter = number_left + (1<<target_qubit)-1
        for counter_right_bits = number_left:end_right_counter
            counter_right_bits_1 = counter_right_bits + (1<<target_qubit)
            psi[counter_right_bits+1], psi[counter_right_bits_1+1]=u*[psi[counter_right_bits+1], psi[counter_right_bits_1+1]]
        end
    end
end



In [ ]:
n=12;
error=0.
u=expm(im*(x->x+x')(randn(2,2)+im*randn(2,2)))

for target_qubit=0:n-1
    psi=random_state(2^n);
    psi_out_2=psi;
    psi_out_1=kron(eye(2^(n-target_qubit-1)),u,eye(2^target_qubit))*psi
    apply_unitary!(psi_out_2, u, target_qubit)
    error+=norm(psi_out_1-psi_out_2)
end
@show error;

In [ ]:
n=14;
u=expm(im*(x->x+x')(randn(2,2)+im*randn(2,2)))
psi=random_state(2^n);
target_qubit=rand(0:n-1)

@time kron(eye(2^(n-target_qubit-1)),u,eye(2^target_qubit))*psi
@time apply_unitary!(psi, u, target_qubit)

In [ ]:
# Esto se basara en la formula 3.2.44 del Sakurai
function apply_kick!(psi, b, target_qubit)
    phi=norm(b)
    b_normalized=b/phi
    sigma_n=sigmas[1]*b_normalized[1]+sigmas[2]*b_normalized[2]+sigmas[3]*b_normalized[3]
    u=eye(2)*cos(phi)-im*sigma_n*sin(phi)
    apply_unitary!(psi, u, target_qubit)
end 

In [ ]:
"""
Esta funcion revisa si el bit bit-esimo de n esta o no prendido. 
"""
function testbit(n, bit)
    ~(n&(1<<bit)==0)
end 

In [ ]:
n=4;
2^4
for i = 0:2^n-1
    y= [bin(i,n),testbit(i, 4),testbit(i, 3),testbit(i, 2),testbit(i, 1),testbit(i, 0),i]
    @show y
end

In [ ]:
# Ising:
J=1.
expJ=exp(im*J)
expJc=conj(expJ)

number_of_qubits=6;
target_qubit_1=2;
target_qubit_2=3;
psi=random_state(2^number_of_qubits)

for i = 0: (1<<number_of_qubits)-1
    if testbit(i,target_qubit_1) $ testbit(i,target_qubit_2)
        psi[i+1]*=expJc
    else
        psi[i+1]*=expJ
    end
end

In [ ]:
function apply_ising!(psi, J, target_qubit_1, target_qubit_2)
    expJ=exp(im*J)
    expJc=conj(expJ)
    for i = 0: length(psi)-1
        if testbit(i,target_qubit_1) $ testbit(i,target_qubit_2)
            psi[i+1]*=expJc
        else
            psi[i+1]*=expJ
        end
    end
end 

In [ ]:
b=[ 1. 1. 0.]
phi=norm(b)
b_normalized=b/phi
sigma_n=sigmas[1]*b_normalized[1]+sigmas[2]*b_normalized[2]+sigmas[3]*b_normalized[3]

u=eye(2)*cos(phi)-im*sigma_n*sin(phi)

* Notacion binaria: Para definir estados de qubits, que por hende tienen dimencion $2^N$, donde $N$ es el numero de qubits, es útil usar notación binaria:

Supongamos que tenemos 3 qubits, entonces una base para el espacio de Hilbert puede ser $|\alpha_1\alpha_2\alpha_3\rangle$, donde $\alpha_i\in\lbrace 0,1 \rbrace$. Entonces, por ejemplo, el estado $|000\rangle$ es representado por el numero "000" en binario, que en decimal es simplemente "0". Y para los demas tenemos algo asi:
$001 \to 1$, $010 \to 2$, $111 \to 7$.. etc.

* Luego, por simplicidad, utilizando la base computacional como nuestra base "canonica", la sustitución es simple:
$|000\rangle \to \left( \begin{array}{c} 1 \\ \vdots \\ 0 \end{array} \right)$, $|001\rangle \to \left( \begin{array}{c} 0 \\ 1 \\ \vdots \end{array} \right)$, etc.

Es decir, la componente que lleva el "1" es precisamente la representacion decimal del estado (comenzando a contar desde cero porsupuesto). Sin embargo en julia los vectores se indexan comenzando de 1, por lo que no tenemos la indexacion tan facil. Sin embargo, podemos conservarla y definir la componente $0 \to N$. Con esta sustitución, todas las componentes se quedan como estan a excepción del cero. Entonces:

$|000\rangle \to \left( \begin{array}{c} 0 \\ \vdots \\ 1 \end{array} \right)$ y $|001\rangle \to \left( \begin{array}{c} 1 \\ 0 \\ \vdots \end{array} \right)$, etc.


Incluir el @time

# Probar que los elementos basicos coinciden con lo que tengo programado en c++

In [ ]:
n=4;
psi=random_state(2^n);
# Escritura, usao de map, real e imag
writedlm("/tmp/estado.dat",map(x ->string(real(x)," ", imag(x)), psi))

In [ ]:
psi_out_julia=psi;
kick=[1. 1.2 1.3];
target_qubit=1
apply_kick!(psi_out_julia, kick, target_qubit);

In [ ]:
command=`/home/carlosp/investigacion/libs/cpp/test_spins -o test_kick_single_spin -q $n --position $target_qubit --bx $(kick[1]) --by $(kick[2]) --bz $(kick[3])`
run(pipeline(command, stdout="/tmp/estado_final.dat"))

In [ ]:
b=readdlm("/tmp/estado_final.dat", ' ', '\n');
psi_out_cpp=[Complex{Float64}((x->x[1]+im*x[2])(b[i,:])) for i in 1:2^n];

In [ ]:
norm(psi_out_julia-psi_out_cpp)

In [ ]:
function compare_kicks(psi, kick, target_qubit)
    writedlm("/tmp/estado.dat",map(x ->string(real(x)," ", imag(x)), psi))
    psi_out_julia=psi;
    apply_kick!(psi_out_julia, kick, target_qubit);
    command=`/home/carlosp/investigacion/libs/cpp/test_spins -o test_kick_single_spin -q $n --position $target_qubit --bx $(kick[1]) --by $(kick[2]) --bz $(kick[3])`
    run(pipeline(command, stdout="/tmp/estado_final.dat"))
    b=readdlm("/tmp/estado_final.dat", ' ', '\n');
    psi_out_cpp=[Complex{Float64}((x->x[1]+im*x[2])(b[i,:])) for i in 1:2^n];
    norm(psi_out_julia-psi_out_cpp)
end

In [ ]:
n=14;
psi=random_state(2^n);
error=0
kick=2*π*rand(3)
for target_qubit=0:n-1
    error+=compare_kicks(psi, kick, target_qubit)
end
error

In [ ]:
n=6;
psia=random_state(2^n);
psit=Complex{Float64}[1, 0, 0 , 0]'';
psi=psia;
# Escritura, usao de map, real e imag
writedlm("/tmp/estado.dat",map(x ->string(real(x)," ", imag(x)), psi))

In [ ]:
psi_out_julia=psi;
J=.1;
target_qubit_1=0
target_qubit_2=1
#apply_ising!(psi_out_julia, J, target_qubit_1, target_qubit_2);


In [ ]:
    command=`/home/carlosp/investigacion/libs/cpp/test_spins -o test_ising_single -q $n --position $target_qubit_1 --position2 $target_qubit_2 --ising_z $J`
    run(pipeline(command, stdout="/tmp/estado_final.dat"))
    b=readdlm("/tmp/estado_final.dat", ' ', '\n');
    psi_out_cpp=[Complex{Float64}((x->x[1]+im*x[2])(b[i,:])) for i in 1:2^n];

In [ ]:
# @show psi_out_cpp;
#@show psi_out_julia;


In [ ]:
function apply_ising!(psi, J, target_qubit_1, target_qubit_2)
    expJ=exp(-im*J)
    expJc=conj(expJ)
    for i = 0: length(psi)-1
        if testbit(i,target_qubit_1) $ testbit(i,target_qubit_2)
            psi[i+1]*=expJc
        else
            psi[i+1]*=expJ
        end
    end
end 
apply_ising!(psi, J, target_qubit_1, target_qubit_2);


In [ ]:
function compare_ising(psi,  J, target_qubit_1, target_qubit_2)
    writedlm("/tmp/estado.dat",map(x ->string(real(x)," ", imag(x)), psi))
    command=`/home/carlosp/investigacion/libs/cpp/test_spins -o test_ising_single -q $n --position $target_qubit_1 --position2 $target_qubit_2 --ising_z $J`
    run(pipeline(command, stdout="/tmp/estado_final.dat"))
    b=readdlm("/tmp/estado_final.dat", ' ', '\n');
    psi_out_cpp=[Complex{Float64}((x->x[1]+im*x[2])(b[i,:])) for i in 1:2^n];
    psi_out_julia=psi;
    apply_ising!(psi_out_julia, J, target_qubit_1, target_qubit_2);
    norm(psi_out_cpp-psi_out_julia)
end

In [ ]:
compare_ising(psi,  J, target_qubit_1, target_qubit_2)

# Cosas por hacer, quiza proyectos finales